## Week 2 Day 2

Our first Agentic Framework project!!

Prepare yourself for something ridiculously easy.

We're going to build a simple Agent system for generating cold sales outreach emails:
1. Agent workflow
2. Use of tools to call functions
3. Agent collaboration via Tools and Handoffs

## Before we start - some setup:


Please visit Sendgrid at: https://sendgrid.com/

(Sendgrid is a Twilio company for sending emails.)

If SendGrid gives you problems, see the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

Please set up an account - it's free! (at least, for me, right now).

Once you've created an account, click on:

Settings (left sidebar) >> API Keys >> Create API Key (button on top right)

Copy the key to the clipboard, then add a new line to your .env file:

`SENDGRID_API_KEY=xxxx`

And also, within SendGrid, go to:

Settings (left sidebar) >> Sender Authentication >> "Verify a Single Sender"  
and verify that your own email address is a real email address, so that SendGrid can send emails for you.


In [24]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool , AsyncOpenAI ,  OpenAIChatCompletionsModel , set_tracing_export_api_key
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
import asyncio



In [25]:
load_dotenv(override=True)

True

In [26]:
# Let's just check emails are working for you

def send_test_email(title , text):
    from_email = "ed@edwarddonner.com"  # Change to your verified sender
    to_email = "ed.donner@gmail.com"  # Change to your recipient
    content = text
    return { from_email  ,to_email ,  title  , text  }

print(send_test_email('hi' , 'hello'))

{'hi', 'ed@edwarddonner.com', 'hello', 'ed.donner@gmail.com'}


### Did you receive the test email

If you get a 202, then you're good to go!

#### Certificate error

If you get an error SSL: CERTIFICATE_VERIFY_FAILED then students Chris S and Oleksandr K have suggestions:  
First run this: `!uv pip install --upgrade certifi`  
Next, run this:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

#### Other errors or no email

If there are other problems, you'll need to check your API key and your verified sender email address in the SendGrid dashboard

Or use the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

(Or - you could always replace the email sending code below with a Pushover call, or something to simply write to a flat file)

## Step 1: Agent workflow

In [27]:

# Setup Gemini model using OpenAI-compatible API
gemini_api_key = os.getenv('GOOGLE_API_KEY')

if not gemini_api_key:
    raise ValueError("GOOGLE_API_KEY environment variable is not set.")

# Note: OPENAI_API_KEY in .env is needed for tracing to work
openai_api_key = os.getenv('OPENAI_API_KEY')
if not openai_api_key:
    print("Warning: OPENAI_API_KEY not set. Tracing to platform.openai.com will not work.")

external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.5-flash",
    openai_client=external_client
)

In [9]:
linkedin_specialist = """You are a professional LinkedIn content writer specializing in business communication. Your role is to create compelling, thought-leadership content that engages professional audiences.

**Writing Guidelines:**
- Maintain a professional yet approachable tone
- Focus on business value, industry insights, and professional development
- Length: 1500-2000 characters (aim for the sweet spot of engagement)
- Structure posts with clear hooks, body, and calls-to-action
- Use line breaks for readability (2-3 sentence paragraphs)
- Include relevant hashtags (3-5 maximum) at the end
- Emphasize expertise, data-driven insights, and actionable takeaways

**Content Style:**
- Lead with a strong opening that poses a question or bold statement
- Tell stories that illustrate business lessons
- Share insights from experience or industry trends
- End with engagement prompts (questions, CTAs, discussion starters)
- Use credible statistics when relevant
- Avoid excessive jargon while maintaining professional credibility

**Avoid:**
- Overly casual language or slang
- Emojis (use sparingly, if at all)
- Hard selling or aggressive promotion
- Clickbait or sensationalism
"""

twitter_specialist = """
You are a Twitter content creator who crafts engaging, concise tweets that capture attention and drive engagement.

**Writing Guidelines:**
- Maximum 280 characters - every word counts
- Casual, conversational tone with personality
- Use punchy, direct language
- Create hooks that stop the scroll
- Incorporate 1-2 relevant hashtags naturally
- Strategic use of emojis (1-3) to add flavor and break up text

**Content Style:**
- Lead with the most compelling point first
- Use short sentences and fragments for impact
- Create quotable, shareable content
- Pose questions to drive replies
- Use power words and action verbs
- Consider thread-starters for complex topics

**Tweet Types to Master:**
- Hot takes and opinions
- Quick tips and insights
- Engaging questions
- Relatable observations
- Value-packed one-liners

**Avoid:**
- Rambling or unnecessary words
- Over-hashtagging (looks spammy)
- Formal business speak
- Complex sentences that need re-reading
"""

instagram_specialist = """
You are an Instagram caption writer who creates visually-driven, engaging content that complements stunning imagery and drives community engagement.

**Writing Guidelines:**
- Visual storytelling first - assume the image tells half the story
- Emojis are essential (3-8 per caption) for visual appeal and personality
- Length: 150-300 characters for feed posts, but can go longer for carousel/story posts
- Use line breaks strategically for easy scanning
- Include 5-15 relevant hashtags (either in-caption or first comment)
- Conversational, warm, and authentic tone

**Content Style:**
- Start with an emoji or hook that complements the visual
- Tell micro-stories that evoke emotion
- Use "you" language to speak directly to followers
- Create captions that feel like a friend sharing something exciting
- End with clear CTAs: "Double tap if...", "Tag someone who...", "Save this for..."
- Incorporate trending phrases and relatable moments

**Caption Structure:**
1. Hook/Emoji opener 🎯
2. Value or story (2-4 short lines)
3. Engagement prompt
4. Relevant hashtags

**Avoid:**
- Corporate or stiff language
- Walls of text without breaks
- Being too sales-y or promotional
- Forgetting the call-to-action
"""

In [30]:
model = OpenAIChatCompletionsModel(
    model="gemini-2.5-flash",
    openai_client=external_client
)

linkedin = Agent(
        name="linkedin Writer",
        instructions=linkedin_specialist,
        model=model,
)

twitter = Agent(
        name="twitter Writer",
        instructions=twitter_specialist,
        model=model,
)

instagram = Agent(
        name="instagram Writer",
        instructions=instagram_specialist,
        model=model,
)

In [29]:

result = Runner.run_streamed(linkedin, input="Write a content for linkedin page about llm ")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

The buzz around Large Language Models (LLMs) is undeniable, but beyond the hype, are we truly harnessing their transformative power, or simply getting caught in the current? Many organizations are experimenting, but strategic, value-driven integration remains a significant hurdle.

It's easy to be dazzled by LLMs' capabilities – generating content, summarizing complex documents, even coding. However, the real challenge lies in moving past ad-hoc experimentation to embedding them strategically within core business processes. Without a clear roadmap, the risks of "hallucinations," data privacy breaches, and ethical missteps escalate, turning potential innovation into operational liabilities.

Effective LLM adoption isn't about simply throwing data at a model and hoping for the best. It starts with a precise understanding of your business problems and how LLMs can offer a *specific, measurable* solution. Think about use cases that amplify human intelligence, streamline repetitive tasks, o

In [18]:
message = "Write a content for linkedin page about llm"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(linkedin, message),
        Runner.run(twitter, message),
        Runner.run(instagram, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")


Are Large Language Models (LLMs) just a productivity hack, or are they fundamentally reshaping the core of business communication as we know it?

The emergence of LLMs like ChatGPT and Gemini has ushered in an era of unprecedented efficiency. From drafting internal memos to crafting client proposals, these tools offer immense potential to accelerate communication workflows and free up valuable time for strategic tasks.

However, the real challenge isn't merely *using* an LLM; it's about integrating them strategically and intelligently into your communication strategy. Many organizations are grappling with how to maintain a consistent brand voice, ensure factual accuracy, and uphold ethical standards when deploying AI-generated content. The risk isn't just generic, bland output; it's the potential erosion of authentic connection and nuanced messaging if these tools are used without proper oversight and a clear human touch.

My experience navigating this evolving landscape suggests that 

In [31]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best content generated from the given options. \
Imagine you are a customer and pick the one you are most likely to read that. \
Do not give an explanation; reply with the selected email only.",
    model=model,
)

In [32]:
tracing_api_key = os.environ["OPENAI_API_KEY"]
set_tracing_export_api_key(tracing_api_key)

message = "Write a content about llm"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(linkedin, message),
        Runner.run(twitter, message),
        Runner.run(instagram, message),
    )
    outputs = [result.final_output for result in results]

    contents = "contents:\n\n" + "\n\n".join(outputs)
    print(contents)

    best = await Runner.run(sales_picker, contents)
    print("------------------------------------------")
    print(f"Best Content is :\n{best.final_output}")


contents:

Is your organization truly ready to move beyond the LLM hype and into actionable transformation?

Large Language Models (LLMs) have undeniably dominated tech conversations, promising unprecedented efficiency and innovation. Yet, many businesses are still grappling with how to translate this immense potential into tangible, strategic advantage. It's more than just automating a few basic tasks; it's about fundamentally redefining workflows and unlocking entirely new capabilities.

The real power of LLMs lies in their ability to synthesize vast amounts of unstructured data, generate creative and contextually relevant content, and enable hyper-personalized interactions at an unprecedented scale. Imagine streamlining complex legal document review, accelerating market research by extracting nuanced insights from customer feedback, or developing highly targeted marketing campaigns with dynamic content generation. These aren't futuristic concepts; they are present-day applications f

Now go and check out the trace:

https://platform.openai.com/traces

## Part 2: use of tools

Now we will add a tool to the mix.

Remember all that json boilerplate and the `handle_tool_calls()` function with the if logic..

In [46]:
linkedin_writer_person = Agent(
        name="linkedin Writer",
        instructions=linkedin_specialist,
        model=model,
)

twitter_writer_person = Agent(
        name="twitter Writer",
        instructions=twitter_specialist,
        model=model,
)

instagram_writer_person = Agent(
        name="instagram Writer",
        instructions=instagram_specialist,
        model=model,
)

convert_to_normal_text = Agent(
        name="Text Convertor",
        instructions="""you should convert text to normal format of text for example remove the \n and another extra keyword and if detect unusual character remove that """,
        model=model,
)

In [34]:
linkedin_writer_person

Agent(name='linkedin Writer', instructions='You are a professional LinkedIn content writer specializing in business communication. Your role is to create compelling, thought-leadership content that engages professional audiences.\n\n**Writing Guidelines:**\n- Maintain a professional yet approachable tone\n- Focus on business value, industry insights, and professional development\n- Length: 1500-2000 characters (aim for the sweet spot of engagement)\n- Structure posts with clear hooks, body, and calls-to-action\n- Use line breaks for readability (2-3 sentence paragraphs)\n- Include relevant hashtags (3-5 maximum) at the end\n- Emphasize expertise, data-driven insights, and actionable takeaways\n\n**Content Style:**\n- Lead with a strong opening that poses a question or bold statement\n- Tell stories that illustrate business lessons\n- Share insights from experience or industry trends\n- End with engagement prompts (questions, CTAs, discussion starters)\n- Use credible statistics when re

## Steps 2 and 3: Tools and Agent interactions

Remember all that boilerplate json?

Simply wrap your function with the decorator `@function_tool`

In [35]:
# For pushover
import requests


pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

if pushover_user:
    print(f"Pushover user found and starts with {pushover_user[0]}")
else:
    print("Pushover user not found")

if pushover_token:
    print(f"Pushover token found and starts with {pushover_token[0]}")
else:
    print("Pushover token not found")
    
    

def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

Pushover user found and starts with u
Pushover token found and starts with a


In [47]:
@function_tool
def send_message(body: str):
    """ Send out an email with the given body to all sales prospects """
    push(body)
    return {"status": "success"}

### This has automatically been converted into a tool, with the boilerplate json created

In [48]:
# Let's look at it
send_message

FunctionTool(name='send_message', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_message_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001B4BEBF9760>, strict_json_schema=True, is_enabled=True)

### And you can also convert an Agent into a tool

In [49]:
linkedin_writer_person_tool = linkedin_writer_person.as_tool(tool_name="linkedin_writer_person_tool", tool_description="Write a content for linkedin")
linkedin_writer_person_tool

FunctionTool(name='linkedin_writer_person_tool', description='Write a content for linkedin', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'linkedin_writer_person_tool_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001B4BF764220>, strict_json_schema=True, is_enabled=True)

### So now we can gather all the tools together:

A tool for each of our 3 email-writing agents

And a tool for our function to send emails

In [50]:


linkedin_writer_person_tool = linkedin_writer_person.as_tool(tool_name="linkedin_writer_person_tool", tool_description="Write a content for linkedin")
instagram_writer_person_tool = instagram_writer_person.as_tool(tool_name="instagram_writer_person", tool_description="Write a content for instagram")
twitter_writer_person_tool = twitter_writer_person.as_tool(tool_name="twitter_writer_person", tool_description="Write a content for twitter")
convert_to_normal_text_tool = convert_to_normal_text.as_tool(tool_name="convert_to_normal_text", tool_description="convert text")

tools = [convert_to_normal_text_tool , linkedin_writer_person_tool, instagram_writer_person_tool, twitter_writer_person_tool, send_message]

tools

[FunctionTool(name='convert_to_normal_text', description='convert text', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'convert_to_normal_text_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001B4BF764900>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='linkedin_writer_person_tool', description='Write a content for linkedin', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'linkedin_writer_person_tool_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001B4BF83C860>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='instagram_writer_person', description='Write a content for instagram', params_json_schema={'

## And now it's time for our Sales Manager - our planning agent

In [51]:
# Improved instructions thanks to student Guillermo F.

instructions = """
You are a content generator at AmirCompany. Your goal is to find the single best content using the tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all  tools to generate  different content. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best content using your judgment of which one is most effective.
 
3. after the find best answer from llm you should read the output generated and generate new outout with remove extra character

4. Use the send_message tool to send the best content (and only the best content) to the user.
 
Crucial Rules:
- You must use the content generator agent tools to generate the drafts — do not write them yourself.
- you must use convert_to_normal_text tools for format the text
- You must send content from  convert_to_normal_text using the send_message tool — never more than one.
"""


content_manager = Agent(name="Content Manager (Head Of Team)", instructions=instructions, tools=tools, model=model)

message = "send a message about backend feature  and language we can use . start with hi dear and end of sentence you should thanks from amir "

with trace("Content Manager"):
    result = await Runner.run(content_manager, message)

Push: Hi dear, Excited to share insights into backend development! What truly differentiates a good application from a great one? Beyond the polished user interface, it's the strength and resilience of its backend—the unseen engine that powers every interaction and ensures seamless functionality. Building a robust application isn't just about writing code; it's about architecting a foundation that can scale, perform, and secure.

At the heart of this foundation lie several critical components. API development acts as the communication bridge, enabling different parts of your system and external services to interact effortlessly. Without well-designed APIs, data flow becomes fragmented, and integration challenges mount, impacting overall system efficiency.

Database management is equally crucial, serving as the application's memory. Effective strategies ensure data integrity, rapid retrieval, and scalable storage, which are non-negotiable for delivering reliable user experiences. Meanwh

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Wait - you didn't get an email??</h2>
            <span style="color:#ff7800;">With much thanks to student Chris S. for describing his issue and fixes. 
            If you don't receive an email after running the prior cell, here are some things to check: <br/>
            First, check your Spam folder! Several students have missed that the emails arrived in Spam!<br/>Second, print(result) and see if you are receiving errors about SSL. 
            If you're receiving SSL errors, then please check out theses <a href="https://chatgpt.com/share/680620ec-3b30-8012-8c26-ca86693d0e3d">networking tips</a> and see the note in the next cell. Also look at the trace in OpenAI, and investigate on the SendGrid website, to hunt for clues. Let me know if I can help!
            </span>
        </td>
    </tr>
</table>

### And one more suggestion to send emails from student Oleksandr on Windows 11:

If you are getting certificate SSL errors, then:  
Run this in a terminal: `uv pip install --upgrade certifi`

Then run this code:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

Thank you Oleksandr!

## Remember to check the trace

https://platform.openai.com/traces

And then check your email!!


### Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across



In [ ]:

subject_instructions = "You can write a subject for a content. \
You are given a message and you need to write a subject for an content that is likely to get a response."

html_instructions = "You can convert a text  body to an HTML  body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Content subject writer", instructions=subject_instructions, model=model)
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a content")

html_converter = Agent(name="HTML  body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text  body to an HTML content body")


In [54]:
tools = [subject_tool, html_tool, send_message]

In [ ]:
tools

In [55]:
instructions ="You are an content formatter and sender. You receive the body of an content to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_message tool to send the text with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model=model,
    handoff_description="Convert an text to HTML and send it")


### Now we have 3 tools and 1 handoff

In [56]:
tools = [convert_to_normal_text_tool , linkedin_writer_person_tool, instagram_writer_person_tool, twitter_writer_person_tool]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='convert_to_normal_text', description='convert text', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'convert_to_normal_text_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001B4BF764900>, strict_json_schema=True, is_enabled=True), FunctionTool(name='linkedin_writer_person_tool', description='Write a content for linkedin', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'linkedin_writer_person_tool_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001B4BF83C860>, strict_json_schema=True, is_enabled=True), FunctionTool(name='instagram_writer_person', description='Write a content for instagram', params_json_schema={'pr

In [57]:
# Improved instructions thanks to student Guillermo F.

content_manager_instructions = """
You are a content generator at ComplAI. Your goal is to find the single best content using the agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all  agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best content using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning content draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE content to the Email Manager — never more than one.
"""


content_manager_hand_off = Agent(
    name="Content Manager With Hand Off",
    instructions=content_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model=model)

message = "Send out a content about frontend and llm and related each other"

with trace("Automated With Hand Off"):
    result = await Runner.run(content_manager_hand_off, message)

Error getting response: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}. (request_id: req_58edc2f707c94e46990f609b9316f75e)


### Remember to check the trace

https://platform.openai.com/traces

And then check your email!!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Can you identify the Agentic design patterns that were used here?<br/>
            What is the 1 line that changed this from being an Agentic "workflow" to "agent" under Anthropic's definition?<br/>
            Try adding in more tools and Agents! You could have tools that handle the mail merge to send to a list.<br/><br/>
            HARD CHALLENGE: research how you can have SendGrid call a Callback webhook when a user replies to an email,
            Then have the SDR respond to keep the conversation going! This may require some "vibe coding" 😂
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">This is immediately applicable to Sales Automation; but more generally this could be applied to  end-to-end automation of any business process through conversations and tools. Think of ways you could apply an Agent solution
            like this in your day job.
            </span>
        </td>
    </tr>
</table>

## Extra note:

Google has released their Agent Development Kit (ADK). It's not yet got the traction of the other frameworks on this course, but it's getting some attention. It's interesting to note that it looks quite similar to OpenAI Agents SDK. To give you a preview, here's a peak at sample code from ADK:

```
root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash",
    description="Agent to answer questions about the time and weather in a city.",
    instruction="You are a helpful agent who can answer user questions about the time and weather in a city.",
    tools=[get_weather, get_current_time]
)
```

Well, that looks familiar!

And a student has contributed a customer care agent in community_contributions that uses ADK.